In [ ]:
!pip install yfinance
!pip install redshift_connector

: 

In [ ]:
import os
import redshift_connector
import yfinance as yf
import pandas as pd
from datetime import datetime

#Connect to the cluster
conn = redshift_connector.connect(
    host = os.environ["REDSHIFT_CODER_HOST"]
    database = os.environ["REDSHIFT_CODER_DB"]
    port = os.environ["REDSHIFT_CODER_PORT"]
    user = os.environ["REDSHIFT_CODER_USER"]
    password = os.environ["REDSHIFT_CODER_PASSWORD"]
)

my_schema = "c_rivasaren_coderhouse"
conn.rollback()
conn.autocommit = True
# Create a Cursor object
cursor = conn.cursor()

#Extract data from Finance API
df_list = []
companies = ['AMZN','GOOG','WMT','TSLA','META'] 
end_date = datetime.now().strftime('%Y-%m-%d')
start_date='2022-01-01'

for x in companies:
    data = yf.Ticker(x)
    data_hist = data.history(start=start_date, end=end_date)
    df_list.append(pd.DataFrame(data_hist).assign(Company=x))

df = pd.concat(df_list)
#rename column name without space
df.rename(columns = {'Stock Splits':'Stock_Splits', 'Open': 'Open_market'}, inplace = True)

#set datetime index as column to insert into redshift table
df = df.reset_index()

#Crea la tabla en redshift (solo ejecutar una vez)
#cursor.execute("create table daily_prices(Date timestamp, Open_market decimal, High decimal, Low decimal, Close decimal, Volume decimal, Dividens decimal, Stock_Splits decimal, Company varchar)")

df[df.Company=='TSLA'].head()

#Delete previous table's data
cursor.execute("TRUNCATE daily_prices")

# Insertar datos en la tabla de Redshift
query_insert = "INSERT INTO daily_prices (Date, Open_market, High, Low, Close, Volume, Dividens, Stock_Splits, Company) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
data_insert = df.values.tolist()
cursor.executemany(query_insert, data_insert)

conn.close()